In [269]:
import tensorflow as tf
import json
import numpy as np
import pandas as pd

from google.cloud import bigquery
from google.cloud import storage

from google.cloud import aiplatform

#https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/custom/custom-tabular-bq-managed-dataset.ipynb

In [270]:
print(tf.__version__)

2.11.0


In [271]:
#!gsutil cp gs://smart-ml-pongthorn/New_Incident.csv data

# !gsutil -m cp -r "gs://tf1-incident-pongthorn/model" .

In [272]:
unUsedColtoPredict=['severity','id','severity_id','severity_name','imported_at']
isLocalModel=True

filePath='../../data/New_Incident.csv'

modelFile_endPoint=1

localFile=True

In [273]:
# def download_table(bq_table_uri: str):
#     # Remove bq:// prefix if present
#     bqclient= bigquery.Client()
#     prefix = "bq://"
#     if bq_table_uri.startswith(prefix):
#         bq_table_uri = bq_table_uri[len(prefix) :]

#     table = bigquery.TableReference.from_string(bq_table_uri)
#     rows = bqclient.list_rows(table)
#     return rows.to_dataframe()

# dfNewData=download_table("pongthorn.SMartML.new_incident")
# dfNewData.to_csv(filePath,index=False)       
# dfNewData.info()


# Load and Map Data Test

In [279]:
if  localFile:
    dfNewData=pd.read_csv(filePath)

# dfNewData=dfNewData.sample(10)
print(dfNewData.info())
# dfNewData

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         43 non-null     int64  
 1   severity_id                43 non-null     int64  
 2   severity_name              43 non-null     object 
 3   sla                        43 non-null     object 
 4   product_type               43 non-null     object 
 5   brand                      43 non-null     object 
 6   service_type               43 non-null     object 
 7   incident_type              43 non-null     object 
 8   open_to_close_hour         43 non-null     float64
 9   response_to_resolved_hour  43 non-null     float64
 10  imported_at                43 non-null     object 
dtypes: float64(2), int64(2), object(7)
memory usage: 3.8+ KB
None


In [280]:
mapping_file="incident_sevirity_to_class.json"

def download_map_severity_class_as_dict():
 
    storage_client = storage.Client()
    buckdfNewData = storage_client.bucket("smart-ml-pongthorn")
    blob = bucket.blob(mapping_file)
    blob.download_to_filename(mapping_file)
    with open(mapping_file, 'r') as file:
            return json.loads(file.read())

if  localFile==False:    
    map_sevirity_to_class= download_map_severity_class_as_dict()
else:
   with open(mapping_file, 'r') as json_file:
     map_sevirity_to_class= json.load(json_file)
                                  
                                  
print(map_sevirity_to_class)


{'Cosmatic': 0, 'Minor': 1, 'Major': 2, 'Critical': 3}


In [281]:
dfNewData.insert(2, 'severity', dfNewData['severity_name'].map(map_sevirity_to_class),True)
print(dfNewData.info())
dfNewData

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         43 non-null     int64  
 1   severity_id                43 non-null     int64  
 2   severity                   43 non-null     int64  
 3   severity_name              43 non-null     object 
 4   sla                        43 non-null     object 
 5   product_type               43 non-null     object 
 6   brand                      43 non-null     object 
 7   service_type               43 non-null     object 
 8   incident_type              43 non-null     object 
 9   open_to_close_hour         43 non-null     float64
 10  response_to_resolved_hour  43 non-null     float64
 11  imported_at                43 non-null     object 
dtypes: float64(2), int64(3), object(7)
memory usage: 4.2+ KB
None


,id,severity_id,severity,severity_name,sla,product_type,brand,service_type,incident_type,open_to_close_hour,response_to_resolved_hour,imported_at
0,2452,1,3,Critical,24x7 4Hrs Response Time,Server,HPE,Incident,Software,4.500000,4.000000,27:09.0
1,2517,2,2,Major,24x7 4Hrs Resolution Time,Server,HPE,Incident,Memory Failure,6.166667,6.000000,27:09.0
2,2519,2,2,Major,24x7 4Hrs Resolution Time,Server,HPE,Incident,Hard Disk Drive Failure,7.166667,7.000000,27:09.0
3,2518,2,2,Major,24x7 4Hrs Resolution Time,Server,HPE,Incident,Hard Disk Drive Failure,7.166667,7.000000,27:09.0
4,2520,2,2,Major,24x7 4Hrs Resolution Time,Server,HPE,Incident,Hard Disk Drive Failure,7.166667,7.000000,27:09.0
5,2168,2,2,Major,24x7 4Hrs Response Time,Server,Cisco,Incident,Fan Failure,2137.500000,2016.883333,27:09.0
6,2490,2,2,Major,24x7 4Hrs Resolution Time,Storage,HPE,Incident,Hard Disk Drive Failure,26.133333,4.700000,27:09.0
7,2479,2,2,Major,24x7 4Hrs Response Time,Storage,NetApp,Incident,Hard Disk Drive Failure,128.983333,0.383333,27:09.0
8,2523,2,2,Major,24x7 4Hrs Response Time,Storage,NetApp,Incident,Hard Disk Drive Failure,8.916667,8.716667,27:09.0
9,2500,3,1,Minor,24x7 4Hrs Response Time,Storage,NetApp,Incident,General Incident,12.066667,1.650000,27:09.0


In [282]:
pdPrediction=pd.DataFrame(columns=['_id','_predict_severity'])
pdPrediction

,_id,_predict_severity


# Load Model from Directory to Predict

In [283]:
if modelFile_endPoint==1 :

    if  isLocalModel: # Window Enviroment
        PATH_FOLDER_ARTIFACTS="model"    
    else: # For running on Vertex AI Notebook t
        PATH_FOLDER_ARTIFACTS="gs://tf1-incident-pongthorn/model"

    model = tf.keras.models.load_model(PATH_FOLDER_ARTIFACTS)    
    print(f"Load from {PATH_FOLDER_ARTIFACTS}")
    # model.summary()

    
    for  row_dict in dfNewData.to_dict(orient="records"):
          incident_id=row_dict['id']
          print(f"{incident_id} - {row_dict['severity']}({row_dict['severity_name']})") 
          for key_removed in unUsedColtoPredict:
           row_dict.pop(key_removed)
          # print(row_dict)  

          input_dict = {name: tf.convert_to_tensor([value]) for name, value in row_dict.items()}


          predictionResult = model.predict(input_dict)
          # print(f"{predictionResult}")   
          # maxResult=np.max(arryResult)
          # maxResult=round( maxResult,4)
          # print(f"{maxResult} of {arryResult}")

          prob = tf.nn.softmax(predictionResult)
          prob_pct=(100 * prob)  
          _class = tf.argmax(predictionResult,-1).numpy()[0]
          
          pdPrediction =pd.concat([pdPrediction,pd.DataFrame.from_dict([{'_id':incident_id, '_predict_severity':_class}])] )
          # print(input_dict)  
            
          print(f"{prob_pct} %   as {_class}")     
          print("======================================================================================")

    dfPredictData=pd.merge(dfNewData,pdPrediction,how='inner',left_on='id',right_on='_id')
    dfPredictData=dfPredictData.drop(columns=['_id'])
    dfPredictData=dfPredictData[['id','_predict_severity','severity','severity_name']]


Load from model
2452 - 3(Critical)
1/1 [==============================] - 0s 284ms/step
[[18.225367 20.05935  43.378807 18.336477]] %   as 2
2517 - 2(Major)
1/1 [==============================] - 0s 42ms/step
[[18.723211 20.157082 38.59516  22.524544]] %   as 2
2519 - 2(Major)
1/1 [==============================] - 0s 29ms/step
[[19.369211 22.834938 32.11249  25.68336 ]] %   as 2
2518 - 2(Major)
1/1 [==============================] - 0s 29ms/step
[[19.369211 22.834938 32.11249  25.68336 ]] %   as 2
2520 - 2(Major)
1/1 [==============================] - 0s 35ms/step
[[19.369211 22.834938 32.11249  25.68336 ]] %   as 2
2168 - 2(Major)
1/1 [==============================] - 0s 32ms/step
[[18.442238 40.703857 22.476002 18.377905]] %   as 1
2490 - 2(Major)
1/1 [==============================] - 0s 41ms/step
[[19.614077 27.600952 31.383127 21.40185 ]] %   as 2
2479 - 2(Major)
1/1 [==============================] - 0s 31ms/step
[[19.560328 29.862621 31.156094 19.420954]] %   as 2
2523 - 2(Maj

# Load Model from Online EndPpint to Predict

In [284]:
# instances = []
# dfX=dfNewData.drop(columns=unUsedColtoPredict)
# for index,row in dfX.iterrows():
#     dict_item=row.to_dict()
#     instance = {}
#     for key, value in dict_item.items():
#         if key in unUsedColtoPredict:
#             continue
#         if value is None:
#             value = ""
#         instance[key] = [value]
#         # instance[key] = value
#     instances.append(instance)

# print(len(instances))
# print(instances)

In [285]:
isEndpointAvaiable=False


try:
    PROJECT_NUMBER="pongthorn"
    ENDPOINT_ID="2704182877817405440"
    REGION="asia-southeast1"
    endpoint = aiplatform.Endpoint(
    endpoint_name=f"projects/{PROJECT_NUMBER}/locations/{REGION}/endpoints/{ENDPOINT_ID}")
    print(endpoint)
    isEndpointAvaiable=True



except Exception as error:
    print(error.message)

resource name: projects/780371717407/locations/asia-southeast1/endpoints/2704182877817405440


In [286]:
if modelFile_endPoint==2 and isEndpointAvaiable:


    for data in dfNewData.to_dict(orient="records"):
        incident_id=data['id']
        severity=data['severity']
        severity_name=data['severity_name']
        #{(a if condition else b): value for key, value in dict.items()}
        request_data = {key : [value] for key,value in data.items() }
        for key_removed in unUsedColtoPredict:
           request_data.pop(key_removed)
        print(request_data)

        response = endpoint.predict([request_data])
        y_predicted = np.argmax(response.predictions, axis=1)
        predictionResult=response[0][0]
        print(predictionResult)
        print(y_predicted)

        pdPrediction =pd.concat([pdPrediction,pd.DataFrame.from_dict([{'_id':incident_id, '_predict_severity':y_predicted[0]}])] )
        print("======================================================================================")


    dfPredictData=pd.merge(dfNewData,pdPrediction,how='inner',left_on='id',right_on='_id')
    dfPredictData=dfPredictData.drop(columns=['_id'])
    dfPredictData=dfPredictData[['id','_predict_severity','severity','severity_name']]


# Classification Rerport

In [287]:
from sklearn.metrics import confusion_matrix,classification_report


In [288]:
dfPredictData

,id,_predict_severity,severity,severity_name
0,2452,2,3,Critical
1,2517,2,2,Major
2,2519,2,2,Major
3,2518,2,2,Major
4,2520,2,2,Major
5,2168,1,2,Major
6,2490,2,2,Major
7,2479,2,2,Major
8,2523,2,2,Major
9,2500,1,1,Minor


In [289]:
className=list(set().union(list(dfPredictData['severity'].unique()),list(dfPredictData['_predict_severity'].unique())))
actualClass=[  f'actual-{x}' for x in  className]
predictedlClass=[  f'predcited-{x}' for x in className]

y_true=list(dfPredictData['severity'])
y_pred=list(dfPredictData['_predict_severity'])
cnf_matrix = confusion_matrix(y_true,y_pred)
cnf_matrix

# #index=actual , column=prediction
cm_df = pd.DataFrame(cnf_matrix,
                     index = actualClass, 
                     columns = predictedlClass)
cm_df

,predcited-0,predcited-1,predcited-2,predcited-3
actual-0,8,1,0,0
actual-1,1,4,5,0
actual-2,0,2,20,0
actual-3,0,1,1,0


In [290]:

print(classification_report(y_true, y_pred, labels=className))

              precision    recall  f1-score   support

           0       0.89      0.89      0.89         9
           1       0.50      0.40      0.44        10
           2       0.77      0.91      0.83        22
           3       0.00      0.00      0.00         2

    accuracy                           0.74        43
   macro avg       0.54      0.55      0.54        43
weighted avg       0.70      0.74      0.72        43



D:\ProgramData\Anaconda3\envs\dev_google\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ProgramData\Anaconda3\envs\dev_google\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ProgramData\Anaconda3\envs\dev_google\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
